In [48]:
import pandas as pd
import altair as alt
import duckdb as db
import altair_saver


In [94]:
parquet_result_1v = '/Users/liammacgaw/Desktop/InfoVis/tp/ResultadosElectorales_1v.parquet'
parquet_result_paso = '/Users/liammacgaw/Desktop/InfoVis/tp/ResultadosElectorales_PASO_2023.parquet'

In [97]:
result_1v = pd.read_parquet('/Users/liammacgaw/Desktop/InfoVis/tp/ResultadosElectorales_1v.parquet')
voto_presidencial_general = db.sql("select * from result_1v where cargo_id = 1").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [177]:
result_paso = pd.read_parquet('/Users/liammacgaw/Desktop/InfoVis/tp/ResultadosElectorales_PASO_2023.parquet')
voto_presidencial_paso = db.sql("select * from result_paso where cargo_id = 1").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [190]:
voto_presidencial_general['distrito_nombre'] = voto_presidencial_general['distrito_nombre'].replace(
    {'Tierra del Fuego, Antártida e Islas del Atlántico Sur': 'Tierra del Fuego'}
)
voto_presidencial_paso['distrito_nombre'] = voto_presidencial_paso['distrito_nombre'].replace(
    {'Tierra del Fuego, Antártida e Islas del Atlántico Sur': 'Tierra del Fuego'}
)
voto_presidencial_general['agrupacion_nombre'] = voto_presidencial_general['agrupacion_nombre'].replace(
    {'LA LIBERTAD AVANZA': 'LLA',
    'JUNTOS POR EL CAMBIO': 'JxC',
    'UNION POR LA PATRIA': 'UP',
    'FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD': 'FIT-U', 
    'HACEMOS POR NUESTRO PAIS': 'HpNP'}
)
voto_presidencial_paso['agrupacion_nombre'] = voto_presidencial_paso['agrupacion_nombre'].replace(
    {'LA LIBERTAD AVANZA': 'LLA',
    'JUNTOS POR EL CAMBIO': 'JxC',
    'UNION POR LA PATRIA': 'UP',
    'FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD': 'FIT-U', 
    'HACEMOS POR NUESTRO PAIS': 'HpNP'}
)

In [248]:
provincia_votos_general = db.sql("select distrito_nombre, agrupacion_nombre, SUM(votos_cantidad) AS total_votos FROM voto_presidencial_general WHERE agrupacion_nombre IN ('UP','LLA','JxC') GROUP BY distrito_nombre, agrupacion_nombre ORDER BY distrito_nombre")
df_provincia_votos_general = provincia_votos_general.df()
provincia_votos_paso = db.sql("select distrito_nombre, agrupacion_nombre, SUM(votos_cantidad) AS total_votos FROM voto_presidencial_paso WHERE agrupacion_nombre IN ('UP','LLA','JxC') GROUP BY distrito_nombre, agrupacion_nombre ORDER BY distrito_nombre")
df_provincia_votos_paso = provincia_votos_paso.df()

In [170]:
colors = {'UP': 'rgb(0, 156, 222)', 'LLA': 'rgb(117, 59, 189)', 'JxC': 'rgb(254, 221, 0)'}

In [239]:
# Cantidad de votos por Agrupacion y Provincia
chart = alt.Chart(df_provincia_votos_general).mark_bar().encode(
    y=alt.Y('distrito_nombre:N', title='Provincia'),
    x=alt.X('total_votos:Q', title='Total de Votos'),
    color=alt.Color('agrupacion_nombre:N', title='Agrupación', scale=alt.Scale(domain=list(colors.keys()), range=list(colors.values()))),
    tooltip=['distrito_nombre:N', 'agrupacion_nombre:N', 'total_votos:Q']
).properties(
    width=1000,
    height=700,
    title='Cantidad de votos por Agrupación y Provincia'
)
chart.save('/Users/liammacgaw/Desktop/InfoVis/tp/votos por provincia.html')

In [209]:
presidente_votos_general = db.sql("select agrupacion_nombre, SUM(votos_cantidad) AS total_votos FROM voto_presidencial_general WHERE agrupacion_nombre IN ('UP','LLA','JxC','FIT-U','HpNP') GROUP BY agrupacion_nombre ORDER BY total_votos DESC")
presidente_votos_paso = db.sql("select agrupacion_nombre, SUM(votos_cantidad) AS total_votos FROM voto_presidencial_paso WHERE agrupacion_nombre IN ('UP','LLA','JxC','FIT-U','HpNP') GROUP BY agrupacion_nombre ORDER BY total_votos DESC")

In [197]:
colors2 = {'UP': 'rgb(0, 156, 222)', 'LLA': 'rgb(117, 59, 189)', 'JxC': 'rgb(254, 221, 0)','FIT-U': 'rgb(249, 84, 97)', 'HpNP': 'rgb(67, 72, 143)'}

In [240]:
df_presidente_votos_general = presidente_votos_general.df()
df_presidente_votos_paso = presidente_votos_paso.df()

In [235]:
# Reultados de las elecciones
paso = alt.Chart(df_presidente_votos_paso).mark_bar().encode(
    y=alt.Y('agrupacion_nombre:N', title='Agrupación', sort=alt.EncodingSortField(field='total_votos:Q', order='descending')),
    x=alt.X('total_votos:Q', title='Total de Votos', scale=alt.Scale(domain=(0, 10000000))), 
    color=alt.Color('agrupacion_nombre:N', title='Agrupación', scale=alt.Scale(domain=list(colors2.keys()), range=list(colors2.values()))),
    tooltip=['agrupacion_nombre:N', 'total_votos:Q']
).properties(
    width=1200,
    height=325,
    title='Total de Votos en Elecciones PASO por Agrupación'
)
generales = alt.Chart(df_presidente_votos_general).mark_bar().encode(
    y=alt.Y('agrupacion_nombre:N', title='Agrupación', sort=alt.EncodingSortField(field='total_votos:Q', order='descending')),
    x=alt.X('total_votos:Q', title='Total de Votos'), 
    color=alt.Color('agrupacion_nombre:N', title='Agrupación', scale=alt.Scale(domain=list(colors2.keys()), range=list(colors2.values()))),
    tooltip=['agrupacion_nombre:N', 'total_votos:Q']
).properties(
    width=1200,
    height=325,
    title='Total de Votos en Elecciones Generales por Agrupación'
)
grafico_final = paso & generales
grafico_final.save('/Users/liammacgaw/Desktop/InfoVis/tp/ambos resultados votos.html')

In [186]:
df_votos_presidente = pd.merge(df_presidente_votos_paso, df_presidente_votos_general, on='agrupacion_nombre', suffixes=('_paso', '_generales'))

In [187]:
df_votos_presidente['diferencia_votos'] = df_votos_presidente['total_votos_generales'] - df_votos_presidente['total_votos_paso']
df_diferencia_votos_presidente = df_votos_presidente[['agrupacion_nombre', 'diferencia_votos']]

In [188]:
colors2 = {'UP': 'rgb(0, 156, 222)', 'LLA': 'rgb(117, 59, 189)', 'JxC': 'rgb(254, 221, 0)','FIT-U': 'rgb(249, 84, 97)', 'HpNP': 'rgb(67, 72, 143)'}

In [221]:
# Diferencia de votos entre elecciones PASO y Generales por Agrupacion
chart = alt.Chart(df_diferencia_votos_presidente).mark_bar().encode(
    y=alt.Y('agrupacion_nombre:N', title='Agrupación'),
    x=alt.X('diferencia_votos:Q', title='Diferencia de Votos'),
    color=alt.Color('agrupacion_nombre:N', title='Agrupación', scale=alt.Scale(domain=list(colors2.keys()), range=list(colors2.values()))),
    tooltip=['agrupacion_nombre:N', 'diferencia_votos:Q']
).properties(
    width=1200,
    height=700,
    title='Diferencia de Votos entre Elecciones PASO y Generales por Agrupación'
)
chart.save('/Users/liammacgaw/Desktop/InfoVis/tp/diferencia de votos.html')

In [258]:
up_provincia_votos_general = db.sql("select distrito_nombre, agrupacion_nombre, SUM(votos_cantidad) AS total_votos FROM voto_presidencial_general WHERE agrupacion_nombre == 'UP' GROUP BY distrito_nombre, agrupacion_nombre ORDER BY total_votos DESC")
df_up_provincia_votos_general = up_provincia_votos_general.df()
df_up_provincia_votos_general = df_up_provincia_votos_general.head(10)

up_provincia_votos_paso = db.sql("select distrito_nombre, agrupacion_nombre, SUM(votos_cantidad) AS total_votos FROM voto_presidencial_paso WHERE agrupacion_nombre == 'UP' GROUP BY distrito_nombre, agrupacion_nombre ORDER BY total_votos DESC")
df_up_provincia_votos_paso = up_provincia_votos_paso.df()
df_up_provincia_votos_paso = df_up_provincia_votos_paso.head(10)

In [305]:
up_paso = alt.Chart(df_up_provincia_votos_paso).mark_bar().encode(
    y=alt.Y('distrito_nombre:N', title='Provincias', sort=alt.EncodingSortField(field='total_votos:Q', order='descending')),
    x=alt.X('total_votos:Q', title='Total de Votos', scale=alt.Scale(domain=(0, 4500000))), 
    color=alt.Color('agrupacion_nombre:N', title='Agrupación', scale=alt.Scale(domain=list(colors.keys()), range=list(colors.values()))),
    tooltip=['distrito_nombre:N', 'agrupacion_nombre:N', 'total_votos:Q']
).properties(
    width=430,
    height=195,
    title='Top 10 provincias con mas votos para Union por la Patria en Elecciones PASO'
)
up_generales = alt.Chart(df_up_provincia_votos_general).mark_bar().encode(
    y=alt.Y('distrito_nombre:N', title='Provincias', sort=alt.EncodingSortField(field='total_votos:Q', order='descending')),
    x=alt.X('total_votos:Q', title='Total de Votos', scale=alt.Scale(domain=(0, 4500000))), 
    color=alt.Color('agrupacion_nombre:N', title='Agrupación', scale=alt.Scale(domain=list(colors.keys()), range=list(colors.values()))),
    tooltip=['distrito_nombre:N', 'agrupacion_nombre:N', 'total_votos:Q']
).properties(
    width=430,
    height=195,
    title='Top 10 provincias con mas votos para Union por la Patria en Elecciones Generales'
)
grafico_up = up_paso | up_generales

In [257]:
lla_provincia_votos_general = db.sql("select distrito_nombre, agrupacion_nombre, SUM(votos_cantidad) AS total_votos FROM voto_presidencial_general WHERE agrupacion_nombre == 'LLA' GROUP BY distrito_nombre, agrupacion_nombre ORDER BY total_votos DESC")
df_lla_provincia_votos_general = lla_provincia_votos_general.df()
df_lla_provincia_votos_general = df_lla_provincia_votos_general.head(10)

lla_provincia_votos_paso = db.sql("select distrito_nombre, agrupacion_nombre, SUM(votos_cantidad) AS total_votos FROM voto_presidencial_paso WHERE agrupacion_nombre == 'LLA' GROUP BY distrito_nombre, agrupacion_nombre ORDER BY total_votos DESC")
df_lla_provincia_votos_paso = lla_provincia_votos_paso.df()
df_lla_provincia_votos_paso = df_lla_provincia_votos_paso.head(10)

In [306]:
lla_paso = alt.Chart(df_lla_provincia_votos_paso).mark_bar().encode(
    y=alt.Y('distrito_nombre:N', title='Provincias', sort=alt.EncodingSortField(field='total_votos:Q', order='descending')),
    x=alt.X('total_votos:Q', title='Total de Votos', scale=alt.Scale(domain=(0, 4500000))), 
    color=alt.Color('agrupacion_nombre:N', title='Agrupación', scale=alt.Scale(domain=list(colors.keys()), range=list(colors.values()))),
    tooltip=['distrito_nombre:N', 'agrupacion_nombre:N', 'total_votos:Q']
).properties(
    width=430,
    height=195,
    title='Top 10 provincias con mas votos para La Libertad Avanza en Elecciones PASO'
)
lla_generales = alt.Chart(df_lla_provincia_votos_general).mark_bar().encode(
    y=alt.Y('distrito_nombre:N', title='Provincias', sort=alt.EncodingSortField(field='total_votos:Q', order='descending')),
    x=alt.X('total_votos:Q', title='Total de Votos', scale=alt.Scale(domain=(0, 4500000))), 
    color=alt.Color('agrupacion_nombre:N', title='Agrupación', scale=alt.Scale(domain=list(colors.keys()), range=list(colors.values()))),
    tooltip=['distrito_nombre:N', 'agrupacion_nombre:N', 'total_votos:Q']
).properties(
    width=430,
    height=195,
    title='Top 10 provincias con mas votos para La Libertad Avanza en Elecciones Generales'
)
grafico_lla = lla_paso | lla_generales

In [256]:
jxc_provincia_votos_general = db.sql("select distrito_nombre, agrupacion_nombre, SUM(votos_cantidad) AS total_votos FROM voto_presidencial_general WHERE agrupacion_nombre == 'JxC' GROUP BY distrito_nombre, agrupacion_nombre ORDER BY total_votos DESC")
df_jxc_provincia_votos_general = jxc_provincia_votos_general.df()
df_jxc_provincia_votos_general = df_jxc_provincia_votos_general.head(10)

jxc_provincia_votos_paso = db.sql("select distrito_nombre, agrupacion_nombre, SUM(votos_cantidad) AS total_votos FROM voto_presidencial_paso WHERE agrupacion_nombre == 'JxC' GROUP BY distrito_nombre, agrupacion_nombre ORDER BY total_votos DESC")
df_jxc_provincia_votos_paso = jxc_provincia_votos_paso.df()
df_jxc_provincia_votos_paso = df_jxc_provincia_votos_paso.head(10)

In [307]:
jxc_paso = alt.Chart(df_jxc_provincia_votos_paso).mark_bar().encode(
    y=alt.Y('distrito_nombre:N', title='Provincias', sort=alt.EncodingSortField(field='total_votos:Q', order='descending')),
    x=alt.X('total_votos:Q', title='Total de Votos', scale=alt.Scale(domain=(0, 4500000))), 
    color=alt.Color('agrupacion_nombre:N', title='Agrupación', scale=alt.Scale(domain=list(colors.keys()), range=list(colors.values()))),
    tooltip=['distrito_nombre:N', 'agrupacion_nombre:N', 'total_votos:Q']
).properties(
    width=430,
    height=195,
    title='Top 10 provincias con mas votos para Juntos Por el Cambio en Elecciones PASO'
)
jxc_generales = alt.Chart(df_jxc_provincia_votos_general).mark_bar().encode(
    y=alt.Y('distrito_nombre:N', title='Provincias', sort=alt.EncodingSortField(field='total_votos:Q', order='descending')),
    x=alt.X('total_votos:Q', title='Total de Votos', scale=alt.Scale(domain=(0, 4500000))), 
    color=alt.Color('agrupacion_nombre:N', title='Agrupación', scale=alt.Scale(domain=list(colors.keys()), range=list(colors.values()))),
    tooltip=['distrito_nombre:N', 'agrupacion_nombre:N', 'total_votos:Q']
).properties(
    width=430,
    height=195,
    title='Top 10 provincias con mas votos para Juntos Por el Cambio en Elecciones Generales'
)
grafico_jxc = jxc_paso | jxc_generales

In [308]:
grafico_final = grafico_up & grafico_lla & grafico_jxc
grafico_final.save('/Users/liammacgaw/Desktop/InfoVis/tp/top provincias por partido.html')